In [ ]:
# install GIS UTILS
# !pip install -i https://test.pypi.org/simple/ gis-utils-pkg-dillhicks==0.0.2

In [21]:
from tqdm import tqdm
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import fiona
import os
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import rasterio
from rasterio.mask import mask
from gis_utils import raster

In [ ]:
def shp_lookup(shp_dir, search_prop, query):
    """
    function that returns name of shapefile (.shp) associated with a particualar field
    If looking for shp by city, use

    Args:
        shp_dir : str
            location of shapefiles to be searched
        
        search_prop : str
            field to search for in shapefiles
            (city: 'NAME_2')
            (region/state: 'NAME_1')
        
        query : str
            string to be searched for i.e. 'Delhi'

    Returns:
        str : filename of shapefile
    """
    for filename in os.listdir(shp_dir):
        if filename.endswith(".shp"):
            with fiona.open(shp_dir + filename) as src:
                if src[0]['properties'][search_prop] == query:
                    return filename
    return "No shapefile exists"

In [1]:
def crop_raster(raster_path, vector_path, out_dir, name=None):
    """
    Crop large single band raster (.tif) given shapefile (.shp) within 
    bounds of raster. NOT optimized for iteration with multiple vector 
    files, large raster should only be loaded once if creating many 
    cropped images.
    
    Args:
        raster_path : str
            location of large raster to be cropped (.tif)
        
        vector_path : str
            location of shapefile to crop around (.shp) 
            (city: 'NAME_2')
            (region/state: 'NAME_1')
        
        out_dir : str
            directory to write cropped file to. i.e. "../out/dir/"
        
        name : str
            name of new file, else will use vector_path with .tif extension

    Returns:
        str : filename of cropped file
        """
    
    # check for proper file extensions
    if raster_path[-4:] != ".tif":
        print("ERROR: No files written or opened, raster must be (.tif)")
        return "ERROR: see console"
    if vector_path[-4:] != ".shp":
        print("ERROR: No files written or opened, vector must be (.shp)")
        return "ERROR: see console"
    
    # specify path to write image
    if name is None:
        base = os.path.basename(raster_path)
        base = base.replace(".tif", ".shp")
        out_path = os.path.join(out_dir, base)
        
    # read vector file (.shp)
    shp_file = gpd.read_file(vector_path)
    
    # Crop raster using vector file
    with rasterio.open(raster_fp) as src:
        cropped_raster, meta = es.crop_image(src, shp_file)

    # Save cropped raster
    with rasterio.open(out_path, 'w', **meta) as dst:
        dst.write(cropped_raster)
    
    return 

In [17]:
## Masking raster with shapefile

# read shapefile
vec_fp = "../../shapefiles/regions/NAME_2_383.shp" # Firozpur
shapefile = gpd.read_file(vec_fp)

# extract the geometry in GeoJSON format
geoms = shapefile.geometry.values # list of shapely geometries
geometry = geoms[0] # shapely geometry

# transform to GeJSON format
from shapely.geometry import mapping
geoms = [mapping(geoms[0])]

In [23]:
# extract the raster values values within the polygon 
with rasterio.open(raster_fp) as src:
     out_image, out_transform = mask(src, geoms, crop=True)